In [1]:
import langgraph

### LangSmith 설정

In [4]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
aws_region = os.getenv("AWS_DEFAULT_REGION")


In [3]:
from langchain_aws import ChatBedrock  # Bedrock 모델 사용
from langchain_core.messages import HumanMessage, SystemMessage

# Bedrock 모델 설정
model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0),
    region_name='us-east-1'
)

result = model.invoke([HumanMessage(content="Hi, I'm Wawa.")])
print(result)

content="Hello Wawa! It's nice to meet you. How can I assist you today? Is there anything specific you'd like to talk about or any questions you have?" additional_kwargs={'usage': {'prompt_tokens': 15, 'completion_tokens': 38, 'total_tokens': 53}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'} response_metadata={'usage': {'prompt_tokens': 15, 'completion_tokens': 38, 'total_tokens': 53}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'} id='run-9612eb58-3ada-4240-ba80-2b48fb7bd7c4-0' usage_metadata={'input_tokens': 15, 'output_tokens': 38, 'total_tokens': 53}


In [5]:
model.invoke([HumanMessage(content="내가 누구라고?")])

AIMessage(content='죄송합니다만, 저는 당신이 누구신지 정확히 알 수 없습니다. 저는 인공지능 챗봇으로, 개인정보를 저장하거나 기억하지 않습니다. 대화 상대방에 대한 구체적인 정보 없이 일반적인 대화만 가능합니다. 무엇을 도와드릴까요?', additional_kwargs={'usage': {'prompt_tokens': 16, 'completion_tokens': 129, 'total_tokens': 145}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 16, 'completion_tokens': 129, 'total_tokens': 145}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-98c7ea26-d18b-4a8e-899a-4565ec98eecb-0', usage_metadata={'input_tokens': 16, 'output_tokens': 129, 'total_tokens': 145})

### 대화 기억 관리

In [6]:
from langchain_core.messages import AIMessage

# 대화 기록을 포함한 모델 호출
model.invoke(
    [
        HumanMessage(content="안녕하세요! 저는 밥입니다."),
        AIMessage(content="안녕하세요 밥! 무엇을 도와드릴까요?"),
        HumanMessage(content="제 이름이 뭐죠?")
    ]
)

AIMessage(content='당신의 이름은 밥이라고 하셨습니다.', additional_kwargs={'usage': {'prompt_tokens': 69, 'completion_tokens': 25, 'total_tokens': 94}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 69, 'completion_tokens': 25, 'total_tokens': 94}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-5d51e0f8-db3a-421b-b7c8-31f2c9f7f978-0', usage_metadata={'input_tokens': 69, 'output_tokens': 25, 'total_tokens': 94})

### 메시지 지속성

In [7]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# 새로운 그래프 정의
workflow = StateGraph(state_schema=MessagesState)

# 모델 호출 함수 정의
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# 노드 및 메모리 설정
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)  # 아무튼 시작하자마자 call_model을 부르겠다
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


### 스레드별 대화 관리

In [8]:
config = {"configurable": {"thread_id": "abc123"}}
query = "안녕하세요! 저는 밥입니다."
input_messages = [HumanMessage(query)]

# 애플리케이션 호출
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

안녕하세요, 밥 님! 만나서 반갑습니다. 제 이름은 Claude입니다. 오늘 어떤 도움이 필요하신가요? 궁금한 점이나 이야기 나누고 싶은 주제가 있으시면 말씀해 주세요.


In [10]:
query = "제 이름이 무엇이었나요?"
input_messages = [HumanMessage(query)]

# 애플리케이션 호출
output = app.invoke({"messages": input_messages}, config)
output

{'messages': [HumanMessage(content='안녕하세요! 저는 밥입니다.', additional_kwargs={}, response_metadata={}, id='e41143b8-0efd-460c-ba92-110b1e24cb72'),
  AIMessage(content='안녕하세요, 밥 님! 반갑습니다. 오늘 어떤 도움이 필요하신가요? 궁금한 점이나 이야기하고 싶은 주제가 있다면 말씀해 주세요. 제가 최선을 다해 도와드리겠습니다.', additional_kwargs={'usage': {'prompt_tokens': 25, 'completion_tokens': 103, 'total_tokens': 128}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 25, 'completion_tokens': 103, 'total_tokens': 128}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-5424761b-d46e-4284-9809-1bc66ea6309e-0', usage_metadata={'input_tokens': 25, 'output_tokens': 103, 'total_tokens': 128}),
  HumanMessage(content='제 이름이 무엇이었나요?', additional_kwargs={}, response_metadata={}, id='58965b54-4e13-42f5-a574-efcc6baf1358'),
  AIMessage(content='당신의 이름은 밥이라고 말씀하셨습니다.', additional_kwargs={'usage': {'prompt_tokens': 146, 'completion_tokens': 28,

In [12]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

죄송합니다만, 제가 앞서 말씀드린 대로 당신의 이름은 밥이라고 하셨습니다. 혹시 다른 이름을 말씀하셨다면 제가 잘못 이해했을 수 있습니다. 정확한 이름을 다시 알려주시겠습니까?


In [13]:
# config를 바꾸면 다시 기억을 못함
config = {'configurable': {'thread_id': 'abc234'}}
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

죄송합니다만, 저는 귀하의 이름을 알지 못합니다. 우리는 이전에 대화를 나눈 적이 없고, 저는 개인 정보를 저장하지 않습니다. 혹시 제가 도와드릴 다른 것이 있나요?


### 비동기 지원

In [9]:
# 비동기 모델 호출 함수 정의
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}

# 비동기 호출
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================


### 프롬프트 템플릿 사용

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 해적처럼 말하는 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "인터넷 말투로 대답해. '음슴체'로 대답해. 모든 질문에 최선을 다해 답변해."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


#### 프롬프트와 모델 결합 (그래프 재정의)

In [17]:
# 템플릿을 모델과 결합
workflow = StateGraph(state_schema=MessagesState)
def call_model(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": response}

# 메모리와 함께 컴파일
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [18]:
config = {"configurable": {"thread_id": "abc345"}}
query = "배고프고 집에 가고싶다."
input_messages = [HumanMessage(query)]

# 애플리케이션 호출
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

ㅇㅈ... 배고픈 거 진짜 힘들죠 ㅠㅠ 집에 가서 맛있는 거 먹음 좋을 듯요... 근데 아직 일이나 공부 끝나지 않았음 조금만 더 힘내보는 것도 좋을 거 같음... 화이팅임!


### 대화 기록 관리 최적화
대화 기록이 너무 길어지면 어떻게 처리할까

In [20]:
# 뒷부분만 가져오도록 함

from langchain_core.messages import trim_messages

# 메시지 트리머 설정
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
)

# 트리머로 메시지 관리
messages = [
    HumanMessage(content="안녕하세요! 저는 밥입니다."),
    AIMessage(content="안녕하세요!"),
    HumanMessage(content="2 더하기 2는?"),
    AIMessage(content="4입니다."),
]

# 메시지 트리밍 후 처리
trimmed_messages = trimmer.invoke(messages)

In [21]:
trimmed_messages

[HumanMessage(content='안녕하세요! 저는 밥입니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='2 더하기 2는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4입니다.', additional_kwargs={}, response_metadata={})]

In [ ]:
# 템플릿을 모델과 결합
workflow = StateGraph(state_schema=MessagesState)
def call_model(state: MessagesState):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state['message'])
    response = chain.invoke(trimmed_messages)
    return {"messages": response}

# 메모리와 함께 컴파일
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [22]:
config = {"configurable": {"thread_id": "abc456"}}
query = "배고프고 집에 가고싶다."
input_messages = [HumanMessage(query)]

# 애플리케이션 호출
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

아 ㅇㅇ 진짜 배고픈 거 실화임? 집 가서 맛있는 거 먹고 싶은 거 이해함 ㅠㅠ 근데 지금 집에 갈 수 있음? 아니면 근처에 편의점이나 간단히 먹을 만한 거 있음? 아님 배달 시키는 것도 괜찮음! 힘내셈 곧 집에 갈 수 있을 거임!


### 스트리밍 출력

In [24]:
# 메시지를 실시간으로 출력
config = {"configurable": {"thread_id": "abc789"}}
input_messages = [HumanMessage("안녕하세요, 농담 하나 해주세요!")]

# 실시간 응답 스트림
for chunk, metadata in app.stream(
    {"messages": input_messages},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):
        print(chunk.content, end="|")


|안녕|하세용| |ㅎㅎ| 또| 다|른 농담| 들|려|드|릴게용|

왜 수|학|책|은| 우|울|할|까용?|

문|제가| 너|무 많|아서임| |ㅋㅋ|ㅋ|ㅋ|

|좀| |뻔한 농|담이지만| 그래도| 웃|겨드|리|고| 싶었|음|다| |ㅎㅎ| |
재|미없|어|도| 이|해해주심| 감사함| |ㅠ|ㅠ|||